In [ ]:
# WEEK 2

df_opt.createOrReplaceTempView("telecom_logs")

spark.sql("""
    SELECT
        customerID,
        tenure,
        MonthlyCharges,
        TotalCharges,
        -- 90-day rolling averages (window func magic!)
        avg(MonthlyCharges) OVER (
            PARTITION BY customerID
            ORDER BY tenure
            ROWS BETWEEN 3 PRECEDING AND CURRENT ROW
        ) as avg_monthly_90d,
        sum(tenure) OVER (PARTITION BY customerID) as total_tenure,
        max(TotalCharges) OVER (PARTITION BY customerID) as max_bill,
        count(*) OVER (PARTITION BY customerID) as session_count,
        Churn
    FROM telecom_logs
""").createOrReplaceTempView("features_raw")

df_features = spark.sql("""
    SELECT
        customerID,
        avg_monthly_90d,
        total_tenure,
        max_bill,
        session_count,
        Churn,
        -- High-value flag (production business logic)
        CASE WHEN TotalCharges > 1000 THEN 1 ELSE 0 END as high_value
    FROM features_raw
    GROUP BY customerID, avg_monthly_90d, total_tenure, max_bill, session_count, Churn, TotalCharges
""").cache()

print(" SQL Features Created:")
df_features.show(10)
df_features.printSchema()
print(f"Features shape: {df_features.count():,} rows")


In [ ]:
# WEEK 2
# SPEC: "Analyze Spark Execution Plan (DAG)"

print("PRODUCTION: Spark DAG Analysis")
print("="*60)

# Physical plan (ASCII graph!)
plan = df_features.explain(True)  # Extended = full graph
print(plan)

print("\n EXECUTION STATS (PRODUCTION):")
print(f"• Partitions: {df_features.rdd.getNumPartitions()}")
print(f"• Feature columns: {len(df_features.columns)}")
print(f"• Rows: {df_features.count():,}")
print(f"• Churn rate: {df_features.select(mean('Churn')).collect()[0][0]:.1%}")

# Tuning
df_tuned = df_features.repartition(20).persist()
print(f"\n Tuned: {df_tuned.rdd.getNumPartitions()} partitions")

print("\n DAG analyzed and the features.")
df_tuned.show(5)



In [ ]:
# WEEK 2
available_numeric = ['avg_monthly_90d', 'total_tenure', 'max_bill', 'session_count', 'high_value']
if 'tenure' in df_tuned.columns: available_numeric = ['tenure'] + available_numeric
if 'MonthlyCharges' in df_tuned.columns: available_numeric = ['MonthlyCharges'] + available_numeric
if 'TotalCharges' in df_tuned.columns: available_numeric = ['TotalCharges'] + available_numeric

print("Safe features:", available_numeric)

df_ml_ready = df_tuned.select("Churn", *available_numeric).na.fill(0).cache()
df_ml_ready.show(5)
print(f"Shape: {df_ml_ready.count()} rows x {len(available_numeric)+1} cols")

